In [1]:
from github import Github
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd 
import numpy as np 

In [2]:
# Set up github access and get members from an organization
pd.set_option('display.max_rows', None)
g = Github("6dec4a753453cdbf66d526d8a646004deea5d11c")
df = pd.DataFrame(columns=['Member'])
memberList = []
for member in g.get_organization("PyGithub").get_members():
    memberList.append(member.login)

In [3]:
# Get all of the member's repositories
df = pd.DataFrame(columns=['Username','Repository','DateCreated','Language', 'Size'])
for member in memberList:
    currentUser = g.get_user(member)
    for repo in currentUser.get_repos():
        new_row = {'Username': member.strip(), 'Repository': repo.name.strip(), 'DateCreated': repo.created_at.year, 'Language':repo.language, 'Size':repo.size, 'Contributors':repo.get_contributors().totalCount}
        df = df.append(new_row, ignore_index=True)

In [4]:
df = df.sort_values(by=['Username', 'DateCreated'])
df = df[(df[['Size','Contributors']] != 0).all(axis=1)]
df = df[df.Language != 'None']
for member in memberList:
    mask = (df['Username'] == member)
    df['DateCreated'][mask] = df['DateCreated'] - (df.loc[df['Username'] == member])['DateCreated'].min()
maxYear = df['DateCreated'].max()

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
pd.set_option('precision', 0)
dataframe_collection = [] 
for year in range(maxYear):
    tempArray = df.loc[df['DateCreated'] == year]
    tempArray = tempArray.drop(['Username', 'Repository',"DateCreated"], axis=1)
    yearArray = pd.DataFrame(columns=['Language','Repositories','Average Size','Contributors'])
    yearArray['Language'] = df.Language.unique()
    uniqueValues = df.Language.unique()
    for language in uniqueValues:
        mask = (yearArray['Language'] == language)
        yearArray['Repositories'][mask] = tempArray.loc[tempArray['Language'] == language].Language.count()
        yearArray['Average Size'][mask] = tempArray.loc[tempArray['Language'] == language].Size.mean()
        yearArray['Contributors'][mask] = tempArray.loc[tempArray['Language'] == language].Contributors.mean()
        yearArray['Average Size'].fillna(0, inplace=True)
        yearArray['Repositories'].fillna(0, inplace=True)
        yearArray['Contributors'].fillna(0, inplace=True)
    dataframe_collection.append(yearArray)

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jackengels1/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jackengels1/opt/miniconda3/lib/py

In [6]:
for dataIndex in range(len(dataframe_collection) - 1):
    dataframe_collection[dataIndex + 1]['Repositories'] = dataframe_collection[dataIndex + 1]['Repositories'] + dataframe_collection[dataIndex]['Repositories']
    dataframe_collection[dataIndex + 1]['Average Size'] = (dataframe_collection[dataIndex + 1]['Average Size'] + dataframe_collection[dataIndex]['Average Size']/2)
    dataframe_collection[dataIndex + 1]['Contributors'] = dataframe_collection[dataIndex + 1]['Contributors'] + dataframe_collection[dataIndex]['Contributors']

In [27]:
# Cleaning up some outliers 
for dataIndex in range(len(dataframe_collection)):
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='None', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Emacs Lisp', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Erlang', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='PHP', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='HCL', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Makefile', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='VimL', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Vue', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Go', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Roff', value=np.nan).dropna()
    dataframe_collection[dataIndex] = dataframe_collection[dataIndex].replace(to_replace='Jupyter Notebook', value=np.nan).dropna()
    dataframe_collection[dataIndex]['Year'] = dataIndex
finalDataBase = pd.concat(dataframe_collection)
finalDataBase = finalDataBase.sort_values(by=['Language'])
languages = finalDataBase.Language.unique()
display(finalDataBase)
languages

,Language,Repositories,Average Size,Contributors,Year
0,C,1,104,1,0
0,C,3,2253,64,5
0,C,9,41947,168,6
0,C,3,4505,64,4
0,C,3,9010,64,3
0,C,9,20973,168,7
0,C,3,18020,64,2
0,C,9,10487,168,8
0,C,1,52,1,1
16,C#,1,117032,118,5


array(['C', 'C#', 'C++', 'CSS', 'HTML', 'Java', 'JavaScript', 'OCaml',
       'Objective-C', 'Python', 'Ruby', 'Rust', 'Shell', 'Swift'],
      dtype=object)

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8e+06,Asia,29,779
1,Afghanistan,1957,9e+06,Asia,30,821
2,Afghanistan,1962,1e+07,Asia,32,853
3,Afghanistan,1967,1e+07,Asia,34,836
4,Afghanistan,1972,1e+07,Asia,36,740
5,Afghanistan,1977,1e+07,Asia,38,786
6,Afghanistan,1982,1e+07,Asia,40,978
7,Afghanistan,1987,1e+07,Asia,41,852
8,Afghanistan,1992,2e+07,Asia,42,649
9,Afghanistan,1997,2e+07,Asia,42,635


In [9]:
# make figure
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

In [10]:
years = np.arange(maxYear)
fig_dict["layout"]["xaxis"] = {"range": [0, 100], "title": "Number of Active Repositories"}
fig_dict["layout"]["yaxis"] = {"title": "Number of Contributors", "range": [0, 400]}
fig_dict["layout"]["hovermode"] = "closest"
fig_dict["layout"]["sliders"] = {
    "args": [
        "transition", {
            "duration": 400,
            "easing": "cubic-in-out"
        }
    ],
    "initialValue": "0",
    "plotlycommand": "animate",
    "values": years,
    "visible": True
}

In [12]:
# fill in most of layout
fig_dict["layout"]["updatemenus"] = [
    {
        "buttons": [
            {
                "args": [None, {"frame": {"duration": 500, "redraw": False},
                                "fromcurrent": True, "transition": {"duration": 300,
                                                                    "easing": "quadratic-in-out"}}],
                "label": "Play",
                "method": "animate"
            },
            {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                  "mode": "immediate",
                                  "transition": {"duration": 0}}],
                "label": "Pause",
                "method": "animate"
            }
        ],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }
]

sliders_dict = {
    "active": 0,
    "yanchor": "top",
    "xanchor": "left",
    "currentvalue": {
        "font": {"size": 20},
        "prefix": "Year:",
        "visible": True,
        "xanchor": "right"
    },
    "transition": {"duration": 300, "easing": "cubic-in-out"},
    "pad": {"b": 10, "t": 50},
    "len": 0.9,
    "x": 0.1,
    "y": 0,
    "steps": []
}

In [ ]:
year = 0
for dataFrame in dataframe_collection:
    dataset_by_year = dataset[dataset["year"] == year]
    dataset_by_year_and_cont = dataset_by_year[
        dataset_by_year["continent"] == continent]

    data_dict = {
        "x": list(dataset_by_year_and_cont["lifeExp"]),
        "y": list(dataset_by_year_and_cont["gdpPercap"]),
        "mode": "markers",
        "text": list(dataset_by_year_and_cont["country"]),
        "marker": {
            "sizemode": "area",
            "sizeref": 200000,
            "size": list(dataset_by_year_and_cont["pop"])
        },
        "name": continent
    }
    fig_dict["data"].append(data_dict)